## Find Homorhythm


#### This function predicts homorhythmic passages in a given piece.
    
The method follows various stages:
- gets **durational ngrams**, and finds passages in which these are the same in **more than two voices at a given offsets**
- gets syllables at every offset, and identifies **passages where more than two voices are singing the same lyrics**
- checks the **number of active voices** (thus eliminating places where some voices have rests)

In [2]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy
import itertools
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


## Import Your Piece

- Here you will want to select the appropriate 'prefix' that identifies the location of your file.
- `'Music_Files/'` is for files in the local notebook; `'https://crimproject.org/mei/'` is for the files on CRIM.
- Then provide the full name (and extension) of your music file, such as `'CRIM_Model_0038.mei'`

In [3]:
# Select a prefix:

# prefix = 'Music_Files/'
prefix = 'https://crimproject.org/mei/'

# Add your filename here

mei_file = 'CRIM_Model_0008.mei'

url = prefix + mei_file

piece = importScore(url)

print(piece.metadata)

Successfully imported https://crimproject.org/mei/CRIM_Model_0008.mei
{'title': 'Ave Maria', 'composer': 'Josquin Des Prés'}


### Find Homorhythm

* The function is extremely simple, and requires no parameters:
* `piece.homorhythm()`
* The resulting data frame show the measures/beats, and offsets
* The method follows various stages:

- gets durational ngrams, and finds passages in which these are the same in more than two voices at a given offsets, thus `number_dur_grams = 1` in the results
- gets syllables at every offset, and identifies passages where more than two voices are singing the same lyrics, thus `active_syll_voices` in the results
- checks the number of active voices (thus eliminating places where some voices have rests), thus `active_voices` in the results





In [5]:
hr = piece.homorhythm()
hr['voice_match'] = hr['active_voices'] == hr['active_syll_voices']
hr.head(10)

,,,active_voices,number_dur_ngrams,active_syll_voices,voice_match
Measure,Beat,Offset,,,,
39,1.0,304.0,3.0,1.0,3.0,True
40,3.0,316.0,4.0,1.0,4.0,True
41,1.0,320.0,4.0,1.0,4.0,True
94,1.0,748.0,4.0,1.0,3.0,False
96,3.0,780.0,3.0,1.0,3.0,True
98,1.0,796.0,4.0,1.0,3.0,False
100,3.0,828.0,3.0,1.0,3.0,True
101,3.0,840.0,3.0,1.0,3.0,True
102,3.0,852.0,3.0,1.0,3.0,True


In [12]:
hr_full = hr[hr['voice_match'] == True]

hr_staggered = hr[hr['voice_match'] == False]
hr_full

,,,active_voices,number_dur_ngrams,active_syll_voices,voice_match
Measure,Beat,Offset,,,,
39,1.0,304.0,3.0,1.0,3.0,True
40,3.0,316.0,4.0,1.0,4.0,True
41,1.0,320.0,4.0,1.0,4.0,True
96,3.0,780.0,3.0,1.0,3.0,True
100,3.0,828.0,3.0,1.0,3.0,True
101,3.0,840.0,3.0,1.0,3.0,True
102,3.0,852.0,3.0,1.0,3.0,True
141,1.0,1184.0,3.0,1.0,3.0,True
143,1.0,1200.0,4.0,1.0,4.0,True


### Here We Explain the Steps

In [4]:
# function for removing non alpha characters from texts

def alpha_only(value):
    if isinstance(value, str):
        return re.sub(r'[^a-zA-Z]', '', value)
    else:
        return value

In [5]:
def find_hr(piece):
    nr = piece.getNoteRest()
    dur = piece.getDuration(df=nr)
    ng = piece.getNgrams(df=dur, n=2)
    dur_ngrams = []
    
    # find passages with more than 2 active voices
    for index, rows in ng.iterrows():

        dur_ngrams_no_nan = [x for x in rows if pd.isnull(x) == False]
        dur_ngrams.append(dur_ngrams_no_nan)

    ng['dur_ngrams'] = dur_ngrams
    # ng['rest_count'] = rests
    ng['active_voices'] = ng['dur_ngrams'].apply(len)
    ng['number_dur_ngrams'] = ng['dur_ngrams'].apply(set).apply(len)
    ng = ng[(ng['number_dur_ngrams'] <2) & (ng['active_voices'] > 2)]
    
    # check rests in multiple parts
    nr.ffill(inplace=True)
    index_of_rests = []
    rests = []
    for index, rows in nr.iterrows():
        rest_test = [y for y in rows if y == "Rest"]
        rests.append(rest_test)
    
    #     index_of_rests.append(index)
    nr["rests"] = rests  
    nr["rests_count"] = nr["rests"].apply(len)
    full_stop = nr[(nr['rests_count'] > 1) ]
    rests_with_mb = piece.detailIndex(full_stop)
    # now get lyric syllables
    lyrics = piece.getLyric()
    lyrics = lyrics.applymap(alpha_only)
    cols = lyrics.columns
    for col in cols:
        lyrics[col] = lyrics[col].str.lower()
    syll_set = []
    for index2, rows2 in lyrics.iterrows():
        syll_no_nan = [z for z in rows2 if pd.isnull(z) == False]
        syll_set.append(syll_no_nan)
    #     print(syll_no_nan)
    lyrics['syllable_set'] = syll_set
    
    # create mask consisting of passages with more than two voices actively singing same syllables
    lyrics['active_syll_voices'] = lyrics['syllable_set'].apply(len)
    # count how _many_ syllables at this offset
    lyrics['number_sylls'] = lyrics['syllable_set'].apply(set).apply(len)
    # get count of possible hr passages (several voices with same syllable)
    lyrics_hr = lyrics[(lyrics['active_syll_voices'] > 2) & (lyrics['number_sylls'] < 2)]
    # piece.detailIndex(lyrics_hr, offset=True)
    # lyrics['is_hr'] = np.where(lyrics['active_voices'] > 3) 
    hr_sylls_mask = lyrics_hr["active_syll_voices"]
    
    # combine results to show passages where more than 2 voices have the same syllables and durations
    ng = ng[['active_voices', "number_dur_ngrams"]]
    hr = pd.merge(ng, hr_sylls_mask, left_index=True, right_index=True)
    result = piece.detailIndex(hr, offset=True)
    return result

In [6]:
find_hr(piece)

active_voices  number_dur_ngrams  active_syll_voices
Measure Beat Offset                                                      
47      4.0  374.0             3.0                1.0                 3.0
48      2.0  378.0             3.0                1.0                 3.0
        3.0  380.0             3.0                1.0                 3.0
49      2.0  386.0             4.0                1.0                 4.0
50      4.0  398.0             5.0                1.0                 5.0
58      1.0  456.0             3.0                1.0                 3.0

In [7]:
# Durations of Notes in Ngrams.  
# "2" is the minimum number to find passages of three HR chords.
nr = piece.getNoteRest()
dur = piece.getDuration(df=nr)
ng = piece.getNgrams(df=dur, n=2)
ng


,Cantus,Altus,Quintus,Tenor,Sextus,Bassus
0.0,"(8.0, 6.0)","(8.0, 6.0)","(32.0, 8.0)","(8.0, 8.0)","(28.0, 4.0)","(32.0, 8.0)"
8.0,"(6.0, 2.0)","(6.0, 2.0)",NaN,"(8.0, 6.0)",NaN,NaN
14.0,"(2.0, 4.0)","(2.0, 4.0)",NaN,NaN,NaN,NaN
16.0,"(4.0, 4.0)","(4.0, 4.0)",NaN,"(6.0, 2.0)",NaN,NaN
20.0,"(4.0, 4.0)","(4.0, 4.0)",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
561.0,NaN,NaN,"(1.0, 4.0)",NaN,"(1.0, 2.0)",NaN
562.0,NaN,NaN,"(4.0, 2.0)",NaN,"(2.0, 4.0)","(2.0, 4.0)"
564.0,NaN,"(3.0, 1.0)",NaN,NaN,"(4.0, 16.0)","(4.0, 16.0)"
566.0,NaN,NaN,"(2.0, 16.0)",NaN,NaN,NaN


In [8]:
dur_ngrams = []

for index, rows in ng.iterrows():

    dur_ngrams_no_nan = [x for x in rows if pd.isnull(x) == False]
    dur_ngrams.append(dur_ngrams_no_nan)

ng['dur_ngrams'] = dur_ngrams
# ng['rest_count'] = rests
ng['active_voices'] = ng['dur_ngrams'].apply(len)
ng['number_dur_ngrams'] = ng['dur_ngrams'].apply(set).apply(len)
ng = ng[(ng['number_dur_ngrams'] <2) & (ng['active_voices'] > 2)]

ng.head()

,Cantus,Altus,Quintus,Tenor,Sextus,Bassus,dur_ngrams,active_voices,number_dur_ngrams
372.0,"(2.0, 2.0)","(2.0, 2.0)",NaN,"(2.0, 2.0)",NaN,NaN,"[(2.0, 2.0), (2.0, 2.0), (2.0, 2.0)]",3,1
374.0,"(2.0, 2.0)","(2.0, 2.0)",NaN,"(2.0, 2.0)",NaN,NaN,"[(2.0, 2.0), (2.0, 2.0), (2.0, 2.0)]",3,1
378.0,"(2.0, 2.0)","(2.0, 2.0)",NaN,"(2.0, 2.0)",NaN,NaN,"[(2.0, 2.0), (2.0, 2.0), (2.0, 2.0)]",3,1
380.0,"(2.0, 2.0)","(2.0, 2.0)",NaN,"(2.0, 2.0)",NaN,NaN,"[(2.0, 2.0), (2.0, 2.0), (2.0, 2.0)]",3,1
386.0,NaN,"(2.0, 2.0)","(2.0, 2.0)",NaN,"(2.0, 2.0)","(2.0, 2.0)","[(2.0, 2.0), (2.0, 2.0), (2.0, 2.0), (2.0, 2.0)]",4,1


In [9]:
# Checks for Rests in All Voices

pd.set_option('display.max_rows', None)
nr.ffill(inplace=True)
index_of_rests = []
rests = []
for index, rows in nr.iterrows():
    rest_test = [y for y in rows if y == "Rest"]
    rests.append(rest_test)
#     index_of_rests.append(index)
nr["rests"] = rests  
nr["rests_count"] = nr["rests"].apply(len)
full_stop = nr[(nr['rests_count'] > 1) ]
rests_with_mb = piece.detailIndex(full_stop)
full_stop.head()

,Cantus,Altus,Quintus,Tenor,Sextus,Bassus,rests,rests_count
0.0,G4,G3,Rest,Rest,Rest,Rest,"[Rest, Rest, Rest, Rest]",4
8.0,B-4,G4,Rest,G3,Rest,Rest,"[Rest, Rest, Rest]",3
14.0,B-4,G4,Rest,G3,Rest,Rest,"[Rest, Rest, Rest]",3
16.0,A4,F4,Rest,D4,Rest,Rest,"[Rest, Rest, Rest]",3
20.0,F4,D4,Rest,D4,Rest,Rest,"[Rest, Rest, Rest]",3


In [10]:
# here we get the syllables sung at any offset
lyrics = piece.getLyric()
lyrics = lyrics.applymap(alpha_only)
cols = lyrics.columns
for col in cols:
    lyrics[col] = lyrics[col].str.lower()
syll_set = []
for index2, rows2 in lyrics.iterrows():
    syll_no_nan = [z for z in rows2 if pd.isnull(z) == False]
    syll_set.append(syll_no_nan)
#     print(syll_no_nan)
lyrics['syllable_set'] = syll_set
lyrics.head()

,Cantus,Altus,Quintus,Tenor,Sextus,Bassus,syllable_set
0.0,ul,ul,NaN,NaN,NaN,NaN,"[ul, ul]"
8.0,ti,ti,NaN,ul,NaN,NaN,"[ti, ti, ul]"
14.0,mi,mi,NaN,NaN,NaN,NaN,"[mi, mi]"
16.0,miei,miei,NaN,ti,NaN,NaN,"[miei, miei, ti]"
20.0,sos,sos,NaN,NaN,NaN,NaN,"[sos, sos]"


In [11]:
# count of voices with syllables at this offset
lyrics['active_syll_voices'] = lyrics['syllable_set'].apply(len)
# count how _many_ syllables at this offset
lyrics['number_sylls'] = lyrics['syllable_set'].apply(set).apply(len)
# get count of possible hr passages (several voices with same syllable)
lyrics_hr = lyrics[(lyrics['active_syll_voices'] > 2) & (lyrics['number_sylls'] < 2)]
# piece.detailIndex(lyrics_hr, offset=True)
# lyrics['is_hr'] = np.where(lyrics['active_voices'] > 3) 
hr_sylls_mask = lyrics_hr["active_syll_voices"]
lyrics_hr
# hr_sylls_mask

,Cantus,Altus,Quintus,Tenor,Sextus,Bassus,syllable_set,active_syll_voices,number_sylls
280.0,NaN,ta,NaN,ta,NaN,ta,"[ta, ta, ta]",3,1
374.0,gi,gi,NaN,gi,NaN,NaN,"[gi, gi, gi]",3,1
376.0,te,te,NaN,te,NaN,NaN,"[te, te, te]",3,1
378.0,ne,ne,NaN,ne,NaN,NaN,"[ne, ne, ne]",3,1
380.0,rat,rat,NaN,rat,NaN,NaN,"[rat, rat, rat]",3,1
382.0,toin,toin,NaN,toin,NaN,NaN,"[toin, toin, toin]",3,1
384.0,ciel,ciel,NaN,ciel,NaN,NaN,"[ciel, ciel, ciel]",3,1
386.0,NaN,gi,gi,NaN,gi,gi,"[gi, gi, gi, gi]",4,1
388.0,NaN,te,te,NaN,te,te,"[te, te, te, te]",4,1
398.0,NaN,a,a,a,a,a,"[a, a, a, a, a]",5,1


In [12]:
# here we merge the syllable mask (where X voices have the same syllable)
# into the DF of the matching durations in 3+ voices
ng = ng[['active_voices', "number_dur_ngrams"]]
hr = pd.merge(ng, hr_sylls_mask, left_index=True, right_index=True)
piece.detailIndex(hr, offset=True)

active_voices  number_dur_ngrams  active_syll_voices
Measure Beat Offset                                                      
47      4.0  374.0             3.0                1.0                 3.0
48      2.0  378.0             3.0                1.0                 3.0
        3.0  380.0             3.0                1.0                 3.0
49      2.0  386.0             4.0                1.0                 4.0
50      4.0  398.0             5.0                1.0                 5.0
58      1.0  456.0             3.0                1.0                 3.0